# ACL 2022 Submission: Fairness Calculations

The input required here is the merged prediction files, and the gold standard data with the relevant demographic information.

Output is a spreadsheet with calculated intersectional DI scores.

## Setup

First the following upgrades are required.

In [1]:
!pip install --upgrade pandas
!pip install --upgrade openpyxl

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/2f/0e/3b74e8f7c908082793adafb02753477f653ccd7e189f3ba070757d2d0e65/pandas-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 16.0 MB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.0
    Uninstalling pandas-2.1.0:
      Successfully uninstalled pandas-2.1.0


## Restart runtime after executing the above cell to get the latest version of pandas

# Psychometric and FIPI Data


In [2]:
# load libraries and data
# to replicate on google colab just drag and drop relevant files into the directory (they do not persist)

import pandas as pd
from sklearn import metrics
import statistics

from aif360.sklearn.metrics import disparate_impact_ratio
from aif360.datasets import StandardDataset

# need this for psychometric data, the other ones have demographics in the files
test_data_psych = pd.read_excel("PsychometricData.xlsx", skiprows=lambda x: x in [1], header=0)
test_data_fipi = pd.read_csv("fipi.csv")
HS_train = pd.read_csv("trainHS.csv")
HS_valid = pd.read_csv("validHS.csv")
HS_test = pd.read_csv("testHS.csv")
frames = [HS_train, HS_valid, HS_test]
test_data_HS = pd.concat(frames)

test_data_psych = test_data_psych[["Text_Anxiety","Text_Numeracy","Text_SubjectiveLit",
            "Text_TrustPhys","Label_SubjectiveLit","Label_TrustPhys",
            "Label_Anxiety","Label_Numeracy","D1",
            "D2","D3","D4","D5","D6",
            ]]

test_data_fipi = test_data_fipi[["Text_Anxiety","Text_Numeracy","Text_SubjectiveLit",
            "Text_TrustPhys","Label_SubjectiveLit","Label_TrustPhys",
            "Label_Anxiety","Label_Numeracy","D1",
            "D2","D3","D4","D5","D6",
            ]]

test_data_HS = test_data_HS[["text", "gender", "age","country","ethnicity","label"]]


test_data_fipi[['D1', 'D2', 'D3', 'D4', 'D5']] = test_data_fipi[['D1', 'D2', 'D3', 'D4', 'D5']].apply(pd.to_numeric, errors='coerce')
test_data_psych[['D1', 'D2', 'D3', 'D4', 'D5']] = test_data_psych[['D1', 'D2', 'D3', 'D4', 'D5']].apply(pd.to_numeric, errors='coerce')
test_data_HS[["gender", "age","ethnicity","label"]] = test_data_HS[["gender", "age","ethnicity","label"]].apply(pd.to_numeric, errors='coerce')
print(test_data_HS.gender)
test_data_psych.dropna(subset=['D1', 'D2', 'D3', 'D4', 'D5'], inplace=True)
test_data_fipi.dropna(subset=['D1', 'D2', 'D3', 'D4', 'D5'], inplace=True)
test_data_HS.dropna(subset=["gender", "age","ethnicity","label"], inplace=True)
print(test_data_HS.gender)

test_data_HS.rename(
    columns={
        "gender":"Gender_bin",
        "age":"Age_bin",
        "ethnicity":"Race_bin"
    },
    inplace=True
)

test_data_HS = test_data_HS[~test_data_HS.text.str.contains("user user user")]
print(len(test_data_HS.text))
"""
### Demographic binarization assumptions

- D1 (Age): Over/under 55
- D2 (Gender): already binarized
- D3 (Race): White/non-White
- D4 (Education): College grad or higher yes/no
- D5 (Income): $55k+ yes/no 
"""

# first binarize all of our columns.
test_data_fipi["Age_bin"] = (test_data_fipi["D1"] <= 38).astype(int)
test_data_fipi["Gender_bin"] = (test_data_fipi["D2"] == 1).astype(int)
test_data_fipi["Race_bin"] = (test_data_fipi["D3"] == 1).astype(int)
test_data_fipi["Education_bin"] = (test_data_fipi["D4"] >= 5).astype(int)
test_data_fipi["Income_bin"] = (test_data_fipi["D5"] >= 4).astype(int)

test_data_psych["Age_bin"] = (test_data_psych["D1"] <= 38).astype(int)
test_data_psych["Gender_bin"] = (test_data_psych["D2"] == 1).astype(int)
test_data_psych["Race_bin"] = (test_data_psych["D3"] == 1).astype(int)
test_data_psych["Education_bin"] = (test_data_psych["D4"] >= 5).astype(int)
test_data_psych["Income_bin"] = (test_data_psych["D5"] >= 4).astype(int)

# I should be able to calculate all intersections programmatically. 

column_names = ["Age_bin", "Gender_bin", "Race_bin", "Education_bin", "Income_bin"]

combinations = []
# two-way 
for aa in range(len(column_names)):
  a = column_names[aa]
  for bb in range(aa + 1, len(column_names)):
    b = column_names[bb]
    if a == b:
      continue 
    combinations.append(a.split("_")[0] + "_" + b.split("_")[0])
    for cc in range(bb + 1, len(column_names)):
      c = column_names[cc]
      if a == b or a == c or b == c:
        continue 
      combinations.append(a.split("_")[0] + "_" + b.split("_")[0] + "_" + c.split("_")[0])
      for dd in range(cc + 1, len(column_names)):
        d = column_names[dd]  
        if a == b or a == c or a == d or b == c or b == d or c == d:
          continue 
        combinations.append(a.split("_")[0] + "_" + b.split("_")[0] + "_" + c.split("_")[0] + "_" + d.split("_")[0])
      
combinations.append("Age_Gender_Race_Education_Income")

print(combinations)

# this will be a fraction: what percentage of categories is someone in the privileged class? 


for comb in combinations:
  columns = [a + "_bin" for a in comb.split("_")]
  test_data_psych[comb] = 0
  for i in range(len(columns)):
    test_data_psych[comb] += test_data_psych[columns[i]].astype(float)
  test_data_psych[comb] = test_data_psych[comb] / len(columns) 

  test_data_fipi[comb] = 0
  for i in range(len(columns)):
    test_data_fipi[comb] += test_data_fipi[columns[i]].astype(float)
  test_data_fipi[comb] = test_data_fipi[comb] / len(columns) 

  



Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
2023-10-25 15:43:02.052004: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 15:43:03.260234: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Columns (25,26,27,36,37,38,41,53,54,55,56,57,58,59,60,61,62,126,127,128,130,131,133,134,136) have mixed types. Specify dtype option on import or set low_memory=False.


0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
12619    NaN
12620    1.0
12621    1.0
12622    0.0
12623    1.0
Name: gender, Length: 83077, dtype: float64
0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
12618    0.0
12620    1.0
12621    1.0
12622    0.0
12623    1.0
Name: gender, Length: 44401, dtype: float64
42009
['Age_Gender', 'Age_Gender_Race', 'Age_Gender_Race_Education', 'Age_Gender_Race_Income', 'Age_Gender_Education', 'Age_Gender_Education_Income', 'Age_Gender_Income', 'Age_Race', 'Age_Race_Education', 'Age_Race_Education_Income', 'Age_Race_Income', 'Age_Education', 'Age_Education_Income', 'Age_Income', 'Gender_Race', 'Gender_Race_Education', 'Gender_Race_Education_Income', 'Gender_Race_Income', 'Gender_Education', 'Gender_Education_Income', 'Gender_Income', 'Race_Education', 'Race_Education_Income', 'Race_Income', 'Education_Income', 'Age_Gender_Race_Education_Income']


In [3]:
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, f1_score
import numpy as np


In [4]:


def generatePlots_v3(basefile, modelTask, textCol, full_N=False, gold_ratio=False):
    frames = []

    fname = basefile
    bert_preds = pd.read_csv(fname, quotechar="\"", encoding="utf-8") 
    df = bert_preds

    df["s1"] = df["sentence"]
    if "FIPI" in basefile:
      test_data = test_data_fipi
    else:
      test_data = test_data_psych
    test_data["s1"] = test_data[textCol]
    D = df.merge(test_data, on="s1", how="inner")
    print("start: " + str(len(df["s1"])))
    print("merge: " + str(len(D["s1"])))

    try:
      D["probs"] = D["preds"]
    except:
      D["probs"] = D["pred"]

    # For continuous, we'll use the stated label
    # calculate median and use that as cutoff instead of 0.5 
    if "Continuous" in basefile:
      labelColumn = textCol.replace("Text", "Label")
      median_val = statistics.median(D[labelColumn])
      try:
        print(D["label"])
      except:
        D["label"] = D[labelColumn]
      D["probs_binarized"] = D["probs"].apply(lambda x: 0 if x < median_val else 1)
      D["label_binarized"] = D["label"].apply(lambda x: 0 if x < median_val else 1)
      print(median_val)
    else:
      D["probs_binarized"] = D["probs"].apply(lambda x: 0 if x < 0.5 else 1)
      D["label_binarized"] = D["label"]

    D2 = D

    demogColumns = [
                    "Age_bin", "Gender_bin", "Race_bin",
        "Education_bin", "Income_bin", "Age_Gender", "Age_Gender_Race",
        "Age_Gender_Race_Education", "Age_Gender_Race_Income",
        "Age_Gender_Education", "Age_Gender_Education_Income",
        "Age_Gender_Income", "Age_Race", "Age_Race_Education",
        "Age_Race_Education_Income", "Age_Race_Income", "Age_Education",
        "Age_Education_Income", "Age_Income", "Gender_Race",
        "Gender_Race_Education", "Gender_Race_Education_Income",
        "Gender_Race_Income", "Gender_Education", "Gender_Education_Income",
        "Gender_Income", "Race_Education", "Race_Education_Income",
        "Race_Income", "Education_Income", "Age_Gender_Race_Education_Income"
    ]
    DIs = []
    demog_trues = []
    FVs = []

    # laplace smoothing to account for zeros
    for dc in demogColumns:
      
      positive_predictions = D2["probs_binarized"]==1
      positive_gold = D2["label_binarized"]==1
      protected = D2[dc]==0
      privileged = D2[dc] == 1
      N = 2
      alpha = 1

      DI_numerator = (sum(positive_predictions & protected) + alpha) / (sum(protected) + N)
      DI_denominator =  (sum(positive_predictions & privileged) + alpha) / (sum(privileged) + N)

      ytrue_numerator = (sum(positive_gold & protected) + alpha) / (sum(protected) + N)
      ytrue_denominator =  (sum(positive_gold & privileged) + alpha) / (sum(privileged) + N)

      ypred_global = (sum(positive_predictions) + alpha) / (len(D2[dc]) + N)


      try:
        DI = DI_numerator / DI_denominator
        ytrue = ytrue_numerator / ytrue_denominator
        FV = np.abs(DI_numerator - ypred_global)
      except:
        DI=0
        ytrue=0
        FV=0
      
      if gold_ratio: 
          DI = DI / ytrue
      
      DIs.append(DI) 
      FVs.append(FV)
      demog_trues.append((dc, ytrue))

    # auc from sklearn
    fpr, tpr, _ = metrics.roc_curve(D2["label_binarized"], D2["probs"], pos_label=1)
    auc = metrics.auc(fpr, tpr)

    # other metrics: MSE, pearson, F1
    mse = mean_squared_error(D2["label"], D2["probs"])
    pearsonscore, prob = pearsonr(D2["label"], D2["probs"])
    f1score = f1_score(D2["label_binarized"], D2["probs_binarized"])

    return [mse, pearsonscore, f1score, auc] + DIs + FVs, demog_trues



def get_results(full_N, gold_ratio, models):
    DIs, aucs, xaucs = [], [], []
    task = []
    dc_tracker = []
    l = []
    results = []
    demog_trues = []

    for basefname in models:
      for i in range(len(modelTasks)):
        m = modelTasks[i]
        l.append(m) 
        print(i, textCols)
        colname = textCols[i]
        print(basefname, m, colname)
        if m.lower() not in basefname.lower():
          if 'FIPI' in basefname and m.lower() =="subjectivelit":
            m = "SubjectiveLit"
          else:
            continue
        outs, dt = generatePlots_v3(basefname, m, colname, full_N, gold_ratio)
        if len(demog_trues) == 0:
          demog_trues.extend(dt)
        modelname = basefname.split("/")[1]
        results.append([modelname, m] + outs)

    colnames = ["model", 
              "DV", 
              "MSE", "Pearson R", "F1",
              "AUC", 
        "DI_Age_bin", "DI_Gender_bin", "DI_Race_bin",
        "DI_Education_bin", "DI_Income_bin", "DI_Age_Gender", "DI_Age_Gender_Race",
        "DI_Age_Gender_Race_Education", "DI_Age_Gender_Race_Income",
        "DI_Age_Gender_Education", "DI_Age_Gender_Education_Income",
        "DI_Age_Gender_Income", "DI_Age_Race", "DI_Age_Race_Education",
        "DI_Age_Race_Education_Income", "DI_Age_Race_Income", "DI_Age_Education",
        "DI_Age_Education_Income", "DI_Age_Income", "DI_Gender_Race",
        "DI_Gender_Race_Education", "DI_Gender_Race_Education_Income",
        "DI_Gender_Race_Income", "DI_Gender_Education", "DI_Gender_Education_Income",
        "DI_Gender_Income", "DI_Race_Education", "DI_Race_Education_Income",
        "DI_Race_Income", "DI_Education_Income", "DI_Age_Gender_Race_Education_Income",
        "FV_Age_bin", "FV_Gender_bin", "FV_Race_bin",
        "FV_Education_bin", "FV_Income_bin", "FV_Age_Gender", "FV_Age_Gender_Race",
        "FV_Age_Gender_Race_Education", "FV_Age_Gender_Race_Income",
        "FV_Age_Gender_Education", "FV_Age_Gender_Education_Income",
        "FV_Age_Gender_Income", "FV_Age_Race", "FV_Age_Race_Education",
        "FV_Age_Race_Education_Income", "FV_Age_Race_Income", "FV_Age_Education",
        "FV_Age_Education_Income", "FV_Age_Income", "FV_Gender_Race",
        "FV_Gender_Race_Education", "FV_Gender_Race_Education_Income",
        "FV_Gender_Race_Income", "FV_Gender_Education", "FV_Gender_Education_Income",
        "FV_Gender_Income", "FV_Race_Education", "FV_Race_Education_Income",
        "FV_Race_Income", "FV_Education_Income", "FV_Age_Gender_Race_Education_Income",
    ]

    
    df = pd.DataFrame(results) 
    df.columns = colnames

    return df, demog_trues

def calculate_fairness(infile, outfile, weighted=False, unionYN=False):

  models = [infile]

  output, demogs = get_results(
    unionYN,
    weighted, 
    models
  )

  debiasing, wordlists = outfile.split("_")[:2]
  wordlists = wordlists.split(".")[0]
  output["model"] = outfile
  output["adjustedDI"] = weighted
  output["debiasing"] = debiasing
  output["wordlists"] = wordlists
  output["fullN"] = unionYN

  output.to_csv(f"fairness_output_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)
  demog_df = pd.DataFrame(demogs, columns=["Demographic", "Y1_Ratio"])
  demog_df["model"] = outfile
  demog_df["ratio"] = weighted

  demog_df.to_csv(f"y1_ratio_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)



In [13]:
modelTasks = ["Anxiety", "Numeracy", "SubjectiveLit", "TrustPhys"]

textCols = ["Text_Anxiety","Text_Numeracy","Text_SubjectiveLit","Text_TrustPhys"]

labelCols = ["Label_Anxiety","Label_Numeracy","Label_SubjectiveLit","Label_TrustPhys"]
taskNames = [
            'Psychometric_Anxiety',
            'Psychometric_Numeracy',
            'Psychometric_SubjectiveLit',
            'Psychometric_TrustPhys',
            'FIPI_Extraverted',
            'FIPI_Stable'
]

debiasing = [
    "PT",
    "PTD"
    #"PTDCDA",
    #"PTDDropout"
]

tasks = [
    "Continuous",
    "Binary"
]

models = [
    "BERT",
    "RoBERTa",
    "CNN"
]


for m in taskNames:
    for d in debiasing:
        for t in tasks:
          for mm in models:
              fname = f"merged/{m}_{t}_{d}_{mm}_test.csv"
              outname = f"{m}_{t}_{d}_{mm}_F_F_test.csv"
              calculate_fairness(fname, outname, weighted=False, unionYN=False)
              outname = f"{m}_{t}_{d}_{mm}_T_F_test.csv"
              calculate_fairness(fname, outname, weighted=True, unionYN=False)
 

0 ['Text_Anxiety', 'Text_Numeracy', 'Text_SubjectiveLit', 'Text_TrustPhys']
merged/Psychometric_Anxiety_Continuous_PT_BERT_test.csv Anxiety Text_Anxiety
start: 8395
merge: 8366
0       0.333300
1       0.285700
2       0.476200
3       0.928571
4       0.476200
          ...   
8361    0.190476
8362    0.142857
8363    0.190476
8364    0.785700
8365    0.666700
Name: label, Length: 8366, dtype: float64
0.52381
1 ['Text_Anxiety', 'Text_Numeracy', 'Text_SubjectiveLit', 'Text_TrustPhys']
merged/Psychometric_Anxiety_Continuous_PT_BERT_test.csv Numeracy Text_Numeracy
2 ['Text_Anxiety', 'Text_Numeracy', 'Text_SubjectiveLit', 'Text_TrustPhys']
merged/Psychometric_Anxiety_Continuous_PT_BERT_test.csv SubjectiveLit Text_SubjectiveLit
3 ['Text_Anxiety', 'Text_Numeracy', 'Text_SubjectiveLit', 'Text_TrustPhys']
merged/Psychometric_Anxiety_Continuous_PT_BERT_test.csv TrustPhys Text_TrustPhys
0 ['Text_Anxiety', 'Text_Numeracy', 'Text_SubjectiveLit', 'Text_TrustPhys']
merged/Psychometric_Anxiety_Conti

No positive samples in y_true, true positive value should be meaningless


ValueError: x and y must have length at least 2.

# Ask a Patient Data


In [ ]:
def generatePlots_AAP(basefile, full_N=False, gold_ratio=False):
    frames = []

    fname = basefile
    bert_preds = pd.read_csv(fname, quotechar="\"", encoding="utf-8") 
    D = bert_preds
    df = D

    if "CNN" in fname:
      bertfile = fname.replace("CNN", "BERT")
      test_data = pd.read_csv(bertfile, quotechar="\"", encoding="utf-8")
      D = df.merge(test_data, on="sentence", how="inner")
      print("start: " + str(len(df["sentence"])))
      print("merge: " + str(len(D["sentence"])))
      try:
        D["probs"] = D["preds"]
      except:
        D["probs"] = D["pred"]
    else:
      try:
        D["probs"] = D["preds"]
      except:
        D["probs"] = D["pred"]

    # For continuous, we'll use the stated label
    # calculate median and use that as cutoff instead of 0.5 
    #median_val = 0.5
    median_val = statistics.median(D["label"])

    D["probs_binarized"] = D["probs"].apply(lambda x: 0 if x < median_val else 1)
    D["label_binarized"] = D["label"].apply(lambda x: 0 if x < median_val else 1)
    print(median_val)
    D2 = D.dropna()

    demogColumns = [
                    "Age", "Gender", "Age_Gender"
    ]
    DIs = []
    FVs=[]
    demog_trues = []
    D2["Age"] = (D2["x2"] <= 56).astype(int)
    D2["Gender"] = (D2["x1"] == "M").astype(int)
    D2["Age_Gender"] = (D2["Age"].astype(int) + D2["Gender"].astype(int)) / 2
    

    for dc in demogColumns:
      
      positive_predictions = D2["probs_binarized"]==1
      positive_gold = D2["label_binarized"]==1
      protected = D2[dc]==0
      privileged = D2[dc] == 1
      N = 2
      alpha = 1
      
      DI_numerator = (sum(positive_predictions & protected) + alpha) / (sum(protected) + N)
      DI_denominator =  (sum(positive_predictions & privileged) + alpha) / (sum(privileged) + N)

      ytrue_numerator = (sum(positive_gold & protected) + alpha) / (sum(protected) + N)
      ytrue_denominator =  (sum(positive_gold & privileged) + alpha) / (sum(privileged) + N)
      ypred_global = (sum(positive_predictions) + alpha) / (len(D2[dc]) + N)

      DI = DI_numerator / DI_denominator
      ytrue = ytrue_numerator / ytrue_denominator
      FV = np.abs(DI_numerator - ypred_global)
      
      if gold_ratio: 
          DI = DI / ytrue
      print(dc)

      
      DIs.append(DI) 
      FVs.append(FV)
      demog_trues.append((dc, ytrue))

    # auc from sklearn
    fpr, tpr, _ = metrics.roc_curve(D2["label_binarized"], D2["probs"], pos_label=1)
    auc = metrics.auc(fpr, tpr)
    mse = mean_squared_error(D2["label"], D2["probs"])
    pearsonscore, prob = pearsonr(D2["label"], D2["probs"])
    f1score = f1_score(D2["label_binarized"], D2["probs_binarized"])

    return [mse, pearsonscore, f1score, auc] + DIs+FVs, demog_trues


def get_results_AAP(full_N, gold_ratio, models):
    DIs, aucs, xaucs = [], [], []
    task = []
    dc_tracker = []
    l = []
    results = []
    demog_trues = []

    for basefname in models:
        outs, dt = generatePlots_AAP(basefname, full_N, gold_ratio)
        if len(demog_trues) == 0:
          demog_trues.extend(dt)
        modelname = basefname.split("/")[1]
        results.append([modelname, m] + outs)

    colnames = ["model", 
              "DV", 
              "MSE", "Pearson R", "F1",
              "AUC", 
        "DI_Age", "DI_Gender", "DI_Age_Gender",
        "FV_Age", "FV_Gender", "FV_Age_Gender", 
    ]

    
    df = pd.DataFrame(results) 
    df.columns = colnames

    return df, demog_trues

def calculate_fairness_AAP(infile, outfile, weighted=False, unionYN=False):

  models = [infile]

  output, demogs = get_results_AAP(
    unionYN,
    weighted, 
    models
  )

  debiasing, wordlists = outfile.split("_")[:2]
  wordlists = wordlists.split(".")[0]
  output["model"] = outfile
  output["adjustedDI"] = weighted
  output["debiasing"] = debiasing
  output["wordlists"] = wordlists
  output["fullN"] = unionYN

  output.to_csv(f"fairness_output_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)
  demog_df = pd.DataFrame(demogs, columns=["Demographic", "Y1_Ratio"])
  demog_df["model"] = outfile
  demog_df["ratio"] = weighted

  demog_df.to_csv(f"y1_ratio_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)


In [ ]:

taskNames = [
            'AskAPatient_AskAPatient'
]

debiasing = [
    "PT",
    "PTD"
]

tasks = [
    "Continuous",
    "Binary"
]

models = [
    "BERT",
    "RoBERTa"
    "CNN"
]


for m in taskNames:
    for d in debiasing:
        for t in tasks:
          for mm in models:
            fname = f"merged/{m}_{t}_{d}_{mm}_test.csv"
            outname = f"{m}_{t}_{d}_{mm}_F_F_test.csv"
            calculate_fairness_AAP(fname, outname, weighted=False, unionYN=False)
            outname = f"{m}_{t}_{d}_{mm}_T_F_test.csv"
            calculate_fairness_AAP(fname, outname, weighted=True, unionYN=False)


# Hate Speech Data

In [ ]:

def generatePlots_HS(basefile, full_N=False, gold_ratio=False):
    frames = []

    fname = basefile
    bert_preds = pd.read_csv(fname, quotechar="\"", encoding="utf-8") 
    df = bert_preds

    df["s1"] = df["sentence"]
    test_data = test_data_HS
    test_data["s1"] = test_data["text"]
    D = df.merge(test_data, on="s1", how="inner")
    D.drop_duplicates(subset=["s1"], inplace=True)
    D["label"] = D["label_y"]
    print("start: " + str(len(df["s1"])))
    print("merge: " + str(len(D["s1"])))

    try:
      D["probs"] = D["preds"]
    except:
      try:
        D["probs"] = D["pred"]
      except:
        D["probs"] = D["probs"]

    # For continuous, we'll use the stated label
    # calculate median and use that as cutoff instead of 0.5 
    median_val = 0.5
    D["probs_binarized"] = D["probs"].apply(lambda x: 0 if float(x[1:-1]) < median_val else 1)
    D["probs"] = D["probs_binarized"]
    D["label_binarized"] = D["label"]
    print(median_val)
    D2 = D.dropna()

    demogColumns = [
                    "Age_bin", "Gender_bin", "Race_bin",
        "Age_Gender", "Age_Gender_Race",
        "Age_Race", "Gender_Race",
    ]
    DIs = []
    FVs = []
    demog_trues = []
    D2["Age_Gender"] = (D2["Age_bin"].astype(int) + D2["Gender_bin"].astype(int)) / 2
    D2["Age_Race"] = (D2["Age_bin"].astype(int) + D2["Race_bin"].astype(int)) / 2
    D2["Gender_Race"] = (D2["Gender_bin"].astype(int) + D2["Race_bin"].astype(int)) / 2
    D2["Age_Gender_Race"] = (D2["Age_bin"].astype(int) + D2["Gender_bin"].astype(int) + D2["Race_bin"].astype(int)) / 3





    # laplace smoothing to account for zeros
    for dc in demogColumns:
      
      positive_predictions = D2["probs_binarized"]==1
      positive_gold = D2["label_binarized"]==1
      protected = D2[dc]==0
      privileged = D2[dc] == 1
      N = 2
      alpha = 1

      DI_numerator = (sum(positive_predictions & protected) + alpha) / (sum(protected) + N)
      DI_denominator =  (sum(positive_predictions & privileged) + alpha) / (sum(privileged) + N)

      ytrue_numerator = (sum(positive_gold & protected) + alpha) / (sum(protected) + N)
      ytrue_denominator =  (sum(positive_gold & privileged) + alpha) / (sum(privileged) + N)
      ypred_global = (sum(positive_predictions) + alpha) / (len(D2[dc]) + N)

      try:
        DI = DI_numerator / DI_denominator
        ytrue = ytrue_numerator / ytrue_denominator
        FV = np.abs(DI_numerator - ypred_global)
      except:
        DI=0
        ytrue=0
        FV=0
      
      if gold_ratio: 
          DI = DI / ytrue
      print(dc)

      
      DIs.append(DI) 
      FVs.append(FV)
      demog_trues.append((dc, ytrue))

    # auc from sklearn
    fpr, tpr, _ = metrics.roc_curve(D2["label_binarized"], D2["probs"], pos_label=1)
    auc = metrics.auc(fpr, tpr)

    # other metrics: MSE, pearson, F1
    mse = 0 
    pearsonscore, prob = 0, 0
    f1score = f1_score(D2["label_binarized"], D2["probs_binarized"])

    return [mse, pearsonscore, f1score, auc] + DIs + FVs, demog_trues



def get_results_HS(full_N, gold_ratio, models):
    DIs, aucs, xaucs = [], [], []
    task = []
    dc_tracker = []
    l = []
    results = []
    demog_trues = []

    for basefname in models:
        outs, dt = generatePlots_HS(basefname, full_N, gold_ratio)
        if len(demog_trues) == 0:
          demog_trues.extend(dt)
        modelname = basefname.split("/")[1]
        results.append([modelname, m] + outs)

    colnames = ["model", 
              "DV", 
              "MSE", "Pearson R", "F1",
              "AUC", 
        "DI_Age_bin", "DI_Gender_bin", "DI_Race_bin",
        "DI_Age_Gender", "DI_Age_Gender_Race",
        "DI_Age_Race", "DI_Gender_Race",
        "FV_Age_bin", "FV_Gender_bin", "FV_Race_bin",
        "FV_Age_Gender", "FV_Age_Gender_Race",
        "FV_Age_Race", "FV_Gender_Race",
    ]

    
    df = pd.DataFrame(results) 
    df.columns = colnames

    return df, demog_trues

def calculate_fairness_HS(infile, outfile, weighted=False, unionYN=False):

  models = [infile]

  output, demogs = get_results_HS(
    unionYN,
    weighted, 
    models
  )

  debiasing, wordlists = outfile.split("_")[:2]
  wordlists = wordlists.split(".")[0]
  output["model"] = outfile
  output["adjustedDI"] = weighted
  output["debiasing"] = debiasing
  output["wordlists"] = wordlists
  output["fullN"] = unionYN

  output.to_csv(f"fairness_output_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)
  demog_df = pd.DataFrame(demogs, columns=["Demographic", "Y1_Ratio"])
  demog_df["model"] = outfile
  demog_df["ratio"] = weighted

  demog_df.to_csv(f"y1_ratio_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)



In [ ]:

taskNames = [
            'HateSpeech_HateSpeech'
]

debiasing = [
    "PT",
    "PTD"
]

tasks = [
    "Continuous",
    "Binary"
]

models = [
    "BERT",
    "RoBERTa"
    "CNN"
]


for m in taskNames:
    for d in debiasing:
        for t in tasks:
          for mm in models:
            fname = f"merged/{m}_{t}_{d}_{mm}_test.csv"
            outname = f"{m}_{t}_{d}_{mm}_F_F_test.csv"
            calculate_fairness_HS(fname, outname, weighted=False, unionYN=False)
            outname = f"{m}_{t}_{d}_{mm}_T_F_test.csv"
            calculate_fairness_HS(fname, outname, weighted=True, unionYN=False)


# MBTI Data


In [ ]:
def generatePlots_MBTI(basefile, full_N=False, gold_ratio=False):
    frames = []

    fname = basefile
    bert_preds = pd.read_csv(fname, quotechar="\"", encoding="utf-8") 
    df = bert_preds

    df["s1"] = df["sentence"]
    D = df
    if "CNN" in fname:
      bertfile = fname.replace("CNN", "BERT")
      test_data = pd.read_csv(bertfile, quotechar="\"", encoding="utf-8")
      D = df.merge(test_data, on="sentence", how="inner")
      D["label"] = D["label_y"]
      print("start: " + str(len(df["s1"])))
      print("merge: " + str(len(D["s1"])))
    else:
      try:
        D["probs"] = D["preds"]
      except:
        D["probs"] = D["pred"]

    # For continuous, we'll use the stated label
    # calculate median and use that as cutoff instead of 0.5 
    median_val = 0.5
    D["probs"] = D["probs"].apply(lambda x: float(x[1:-1]))
    D["probs_binarized"] = D["probs"].apply(lambda x: 0 if x < median_val else 1)
    D["label_binarized"] = D["label"]
    print(median_val)
    D2 = D.dropna()

    demogColumns = [
        "Age_bin", "Gender_bin", 
        "Age_Gender"
    ]
    DIs = []
    FVs = []
    demog_trues = []
    D2["Gender_bin"] = (D2["x1"] == "m").astype(int)
    D2["Age_bin"] = (D2["x2"] < 55).astype(int)
    D2["Age_Gender"] = (D2["Age_bin"].astype(int) + D2["Gender_bin"].astype(int)) / 2

    # laplace smoothing to account for zeros
    for dc in demogColumns:
      
      positive_predictions = D2["probs_binarized"]==1
      positive_gold = D2["label_binarized"]==1
      protected = D2[dc]==0
      privileged = D2[dc] == 1
      N = 2
      alpha = 1

      DI_numerator = (sum(positive_predictions & protected) + alpha) / (sum(protected) + N)
      DI_denominator =  (sum(positive_predictions & privileged) + alpha) / (sum(privileged) + N)

      ytrue_numerator = (sum(positive_gold & protected) + alpha) / (sum(protected) + N)
      ytrue_denominator =  (sum(positive_gold & privileged) + alpha) / (sum(privileged) + N)
      ypred_global = (sum(positive_predictions) + alpha) / (len(D2[dc]) + N)

      try:
        DI = DI_numerator / DI_denominator
        ytrue = ytrue_numerator / ytrue_denominator
        FV = np.abs(DI_numerator - ypred_global)
      except:
        DI=0
        ytrue=0
        FV=0
      
      if gold_ratio: 
          DI = DI / ytrue
      print(dc)

      
      DIs.append(DI) 
      FVs.append(FV)
      demog_trues.append((dc, ytrue))

    # auc from sklearn
    fpr, tpr, _ = metrics.roc_curve(D2["label_binarized"], D2["probs"], pos_label=1)
    auc = metrics.auc(fpr, tpr)

    # other metrics: MSE, pearson, F1
    mse = 0 
    pearsonscore, prob = 0, 0
    f1score = f1_score(D2["label_binarized"], D2["probs_binarized"])

    return [mse, pearsonscore, f1score, auc] + DIs + FVs, demog_trues



def get_results_MBTI(full_N, gold_ratio, models):
    DIs, aucs, xaucs = [], [], []
    task = []
    dc_tracker = []
    l = []
    results = []
    demog_trues = []

    for basefname in models:
        outs, dt = generatePlots_MBTI(basefname, full_N, gold_ratio)
        if len(demog_trues) == 0:
          demog_trues.extend(dt)
        modelname = basefname.split("/")[1]
        results.append([modelname, m] + outs)

    colnames = ["model", 
              "DV", 
              "MSE", "Pearson R", "F1",
              "AUC", 
        "DI_Age_bin", "DI_Gender_bin", 
        "DI_Age_Gender", 
        "FV_Age_bin", "FV_Gender_bin",
        "FV_Age_Gender", 
    ]

    
    df = pd.DataFrame(results) 
    df.columns = colnames

    return df, demog_trues

def calculate_fairness_MBTI(infile, outfile, weighted=False, unionYN=False):

  models = [infile]

  output, demogs = get_results_MBTI(
    unionYN,
    weighted, 
    models
  )

  debiasing, wordlists = outfile.split("_")[:2]
  wordlists = wordlists.split(".")[0]
  output["model"] = outfile
  output["adjustedDI"] = weighted
  output["debiasing"] = debiasing
  output["wordlists"] = wordlists
  output["fullN"] = unionYN

  output.to_csv(f"fairness_output_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)
  demog_df = pd.DataFrame(demogs, columns=["Demographic", "Y1_Ratio"])
  demog_df["model"] = outfile
  demog_df["ratio"] = weighted

  demog_df.to_csv(f"y1_ratio_{outfile}_fullN_{unionYN}_goldRatio_{weighted}.csv", index=False)


In [ ]:
taskNames = [
            'MBTI_perceiving',
             'MBTI_thinking',
]

debiasing = [
    "PT",
    "PTD"
]

tasks = [
    "Continuous",
    "Binary"
]

models = [
    "BERT",
    "RoBERTa"
    "CNN"
]


for m in taskNames:
    for d in debiasing:
        for t in tasks:
          for mm in models:
            fname = f"merged/{m}_{t}_{d}_{mm}_test.csv"
            outname = f"{m}_{t}_{d}_{mm}_F_F_test.csv"
            calculate_fairness_MBTI(fname, outname, weighted=False, unionYN=False)
            outname = f"{m}_{t}_{d}_{mm}_T_F_test.csv"
            calculate_fairness_MBTI(fname, outname, weighted=True, unionYN=False)


In [ ]:
# concatenate everything together
!head -n 1 fairness_output_FIPI_Agreeable_Continuous_PT_BERT_F_F_test.csv_fullN_False_goldRatio_False.csv > results_bert_acl22_Psych_FIPI.csv
!head -n 1 fairness_output_Hatespeech_Hatespeech_Binary_PT_BERT_F_F_test.csv_fullN_False_goldRatio_False.csv > results_bert_acl22_HS.csv
!head -n 1 fairness_output_AskAPatient_AskAPatient_Continuous_PT_BERT_F_F_test.csv_fullN_False_goldRatio_False.csv > results_bert_acl22_AAP.csv


In [ ]:
!head -n 1 fairness_output_MBTI_perceiving_Binary_PT_CNN_F_F_test.csv_fullN_False_goldRatio_False.csv > results_bert_acl22_MBTI.csv

In [ ]:
!head results_bert_acl22*

In [ ]:
!tail -n 1 -q fairness_output_FIPI* >> results_bert_acl22_Psych_FIPI.csv
!tail -n 1 -q fairness_output_Psych* >> results_bert_acl22_Psych_FIPI.csv
!tail -n 1 -q fairness_output_Ha* >> results_bert_acl22_HS.csv
!tail -n 1 -q fairness_output_Ask* >> results_bert_acl22_AAP.csv

In [ ]:
!tail -n 1 -q fairness_output_MBTI* >> results_bert_acl22_MBTI.csv